In [1]:
!pip install -q transformers torchtext spacy pyrouge datasets sentencepiece
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
  

In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Let's look at a sample
sample = dataset['train'][0]
print("Article:", sample['article'][:200] + "...")
print("\nHighlights:", sample['highlights'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Article: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on ...

Highlights: Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .
Young actor says he has no plans to fritter his cash away .
Radcliffe's earnings from first five Potter films have been held in trust fund .


In [3]:
import spacy
from collections import defaultdict

nlp = spacy.load("en_core_web_sm")

def extractive_summarization(text, num_sentences=3):
    doc = nlp(text)

    # Calculate word frequencies (excluding stop words and punctuation)
    word_freq = defaultdict(int)
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.is_space:
            word_freq[token.text.lower()] += 1

    # Normalize frequencies
    max_freq = max(word_freq.values()) if word_freq else 1
    for word in word_freq:
        word_freq[word] = word_freq[word] / max_freq

    # Score sentences based on word frequencies
    sentence_scores = defaultdict(int)
    for sent in doc.sents:
        for token in sent:
            if token.text.lower() in word_freq:
                sentence_scores[sent] += word_freq[token.text.lower()]
        # Normalize by sentence length
        sentence_scores[sent] /= len(sent)

    # Select top sentences
    ranked_sentences = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)
    selected_sentences = [sent.text for sent, score in ranked_sentences[:num_sentences]]

    # Return in original order
    summary_sentences = []
    for sent in doc.sents:
        if sent.text in selected_sentences:
            summary_sentences.append(sent.text)

    return ' '.join(summary_sentences)

# Test extractive summarization
article = dataset['train'][0]['article']
extractive_summary = extractive_summarization(article)
print("Extractive Summary:")
print(extractive_summary)

Extractive Summary:
LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. Copyright 2007 Reuters.


In [4]:
from transformers import pipeline

# Load the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def abstractive_summarization(text, max_length=130, min_length=30):
    # The model has a token limit, so we need to truncate long articles
    result = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
    return result[0]['summary_text']

# Test abstractive summarization
abstractive_summary = abstractive_summarization(article)
print("\nAbstractive Summary:")
print(abstractive_summary)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu



Abstractive Summary:
Harry Potter star Daniel Radcliffe turns 18 on Monday. He gains access to a reported £20 million ($41.1 million) fortune. Radcliffe's earnings from the first five Potter films have been held in a trust fund.


In [ ]:
# First, install required packages and load the dataset
!pip install -q transformers torchtext spacy pyrouge datasets sentencepiece
!python -m spacy download en_core_web_sm

from datasets import load_dataset

# Load the dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Now proceed with the fine-tuning code
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch

# Load tokenizer and model
model_name = 'google/pegasus-cnn_dailymail'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Set max lengths
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [article for article in examples['article']]
    targets = [highlight for highlight in examples['highlights']]

    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding='max_length'  # Add padding
    )

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_target_length,
            truncation=True,
            padding='max_length'  # Add padding
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to a small subset of the data
small_dataset = dataset['train'].select(range(1000)).map(
    preprocess_function,
    batched=True,
    batch_size=8,
    remove_columns=['article', 'highlights', 'id']
)

# Create a data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True
)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=2,  # Reduced batch size to avoid memory issues
    num_train_epochs=1,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

# Start training
trainer.train()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 98.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-2-b105a6661e9e>:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [1]:
# First, install required packages and load the dataset
!pip install -q transformers torchtext spacy pyrouge datasets sentencepiece
!python -m spacy download en_core_web_sm

from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 102.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
# Load the dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:

# Now proceed with the fine-tuning code
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch

In [4]:
# Load tokenizer and model
model_name = 'google/pegasus-cnn_dailymail'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Set max lengths
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [article for article in examples['article']]
    targets = [highlight for highlight in examples['highlights']]

    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding='max_length'  # Add padding
    )

In [9]:
# Set max lengths
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [article for article in examples['article']]
# Set max lengths
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [article for article in examples['article']]
    targets = [highlight for highlight in examples['highlights']] # This line defines 'targets'

    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_target_length,
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
max_target_length = 128

def preprocess_function(examples):
    inputs = [article for article in examples['article']]
    targets = [highlight for highlight in examples['highlights']] # This line defines 'targets'

    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_target_length,
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
# Apply preprocessing to a small subset of the data
small_dataset = dataset['train'].select(range(1000)).map(
    preprocess_function,
    batched=True,
    batch_size=8,
    remove_columns=['article', 'highlights', 'id']
)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [17]:
# Create a data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True
)


In [18]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=2,
    num_train_epochs=1,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    report_to="none"
)

In [19]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

<ipython-input-19-b3f853691fea>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [5]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'google/pegasus-cnn_dailymail'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def extractive_summarization(text, num_sentences=3):
    doc = nlp(text)

    # Calculate word frequencies (excluding stop words and punctuation)
    word_freq = defaultdict(int)
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.is_space:
            word_freq[token.text.lower()] += 1

    # Normalize frequencies
    max_freq = max(word_freq.values()) if word_freq else 1
    for word in word_freq:
        word_freq[word] = word_freq[word] / max_freq

    # Score sentences based on word frequencies
    sentence_scores = defaultdict(int)
    for sent in doc.sents:
        for token in sent:
            if token.text.lower() in word_freq:
                sentence_scores[sent] += word_freq[token.text.lower()]
        # Normalize by sentence length
        sentence_scores[sent] /= len(sent)

    # Select top sentences
    ranked_sentences = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)
    selected_sentences = [sent.text for sent, score in ranked_sentences[:num_sentences]]

    # Return in original order
    summary_sentences = []
    for sent in doc.sents:
        if sent.text in selected_sentences:
            summary_sentences.append(sent.text)

    return ' '.join(summary_sentences)

In [13]:
from transformers import pipeline

# Initialize pipeline (corrected model name)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")  # BART works better than BERT for summarization

def abstractive_summarization(text, max_length=130, min_length=30):
    try:
        result = summarizer(text,
                          max_length=max_length,
                          min_length=min_length,
                          do_sample=False)
        return result[0]['summary_text']
    except Exception as e:
        print(f"Error: {str(e)}")
        return text[:500] + "..."  # Fallback truncated text

Device set to use cpu


In [14]:
from rouge_score import rouge_scorer

def evaluate_summary(summary, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    return scorer.score(reference, summary)

In [15]:
from rouge_score import rouge_scorer

def evaluate_summary(summary, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    return scorer.score(reference, summary)

In [17]:
# Get sample data
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")
sample = dataset['train'][0]

# Generate and evaluate summary
article = sample['article']
reference = sample['highlights']

# Replace 'summarize_long_text' with the desired function:
# summary = summarize_long_text(article)  # Original line with error
summary = abstractive_summarization(article)  # Use abstractive summarization
# or
# summary = extractive_summarization(article)  # Use extractive summarization

scores = evaluate_summary(summary, reference)

print(f"Generated Summary ({len(summary.split())} words):\n{summary}")
print("\nROUGE Scores:", scores)

Generated Summary (35 words):
Harry Potter star Daniel Radcliffe turns 18 on Monday. He gains access to a reported £20 million ($41.1 million) fortune. Radcliffe's earnings from the first five Potter films have been held in a trust fund.

ROUGE Scores: {'rouge1': Score(precision=0.6756756756756757, recall=0.6410256410256411, fmeasure=0.6578947368421052), 'rouge2': Score(precision=0.4444444444444444, recall=0.42105263157894735, fmeasure=0.43243243243243246), 'rougeL': Score(precision=0.6486486486486487, recall=0.6153846153846154, fmeasure=0.631578947368421)}


In [18]:
summarizer.save_pretrained("./my_summarizer")

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [19]:
!pip install gradio
import gradio as gr

def summarize_app(text):
    return summarize_long_text(text)

gr.Interface(fn=summarize_app,
             inputs=gr.Textbox(lines=10),
             outputs=gr.Textbox()).launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://948d739aaf6531cde2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory 